In [1]:
import matplotlib.pyplot as plt

from fitting_uncertainty import create_ensemble, load_data, create_datasets
import torch

tkwargs = {"dtype": torch.float}
scales = [0.90, 0.95, 1.0, 1.05, 1.1]

emittances = []
betas = []
alphas = []

losses =[]

for scale in scales:
    semits = []
    sbetas = []
    salphas = []
    for ensemble_index in [0,1,2,3,4]:
        #print(scale)
        tkwargs = {"dtype": torch.float}
        save_dir = f"uncertainty/ensemble_mse_scale_{scale}"
        quad_strengths, image_data, bins, xx = load_data(tkwargs)
        train_dset = torch.load("uncertainty/train.dset")
        test_dset = torch.load("uncertainty/test.dset")
        losses += [torch.load(save_dir + "/loss_log.pt")]

        bin_width = bins[1] - bins[0]
        bandwidth = bin_width / 2
        ensemble = create_ensemble(bins, bandwidth)

        from torchensemble.utils import io
        io.load(ensemble, save_dir)

        n_particles = 100
        #ensemble_index = -1
        ensemble[ensemble_index].beam.set_base_beam(
            ensemble[ensemble_index].beam.base_dist,
            n_particles,
            p0c=torch.tensor(63.0e6)
        )

        ensemble.cuda();

        initial_beam = ensemble[ensemble_index].beam()

        # propagate particles w/quad off
        initial_y = initial_beam.y.cpu().detach().numpy()*1e3
        initial_py = initial_beam.py.cpu().detach().numpy()*1e3

        # distances
        dist_to_slits = 3.38 - 2.84 + 0.12/2.0
        dist_to_screen = 3.38 + 0.12/2.0

        # particles
        slits_y = initial_y + initial_py * dist_to_slits
        slits_py = initial_py

        screen_y = initial_y + initial_py * dist_to_screen
        screen_py = initial_py

        cov = torch.cov(initial_beam.data.T)
        ycov = cov[2:4,2:4]

        emit = torch.det(ycov).sqrt()
        twiss = ycov / emit
        #print(f"beta {twiss[0,0]}")
        #print(f"alpha {-twiss[1,0]}")
        #print(f"gamma {twiss[1,1]}")

        # geometric emittance
        #print(f"geo_emit: {emit}")
        print(f"norm emit: {emit*63.0/0.511}")
        
        semits.append(emit*63.0/0.511)
        sbetas.append(twiss[0,0])
        salphas.append(-twiss[1,0])
    emittances.append(semits)
    betas.append(sbetas)
    alphas.append(salphas)

emittances = torch.tensor(emittances)
betas = torch.tensor(betas)
alphas = torch.tensor(alphas)

C:\ProgramData\Miniconda3\envs\phase_space_reconstruction\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


norm emit: 4.039879968331661e-06
norm emit: 4.722669928014511e-06
norm emit: 5.203883119975217e-06
norm emit: 5.090445938549237e-06
norm emit: 4.970832833350869e-06
norm emit: 4.355186320026405e-06
norm emit: 5.315515409165528e-06
norm emit: 6.2953849919722416e-06
norm emit: 5.24425695402897e-06
norm emit: 4.767705377162201e-06
norm emit: 3.701356718011084e-06
norm emit: 4.414522209117422e-06
norm emit: 4.718887339549838e-06
norm emit: 3.85908651878708e-06
norm emit: 4.303900368540781e-06
norm emit: 3.996551640739199e-06
norm emit: 4.586473551171366e-06
norm emit: 6.106324690335896e-06
norm emit: 4.1754424273676705e-06
norm emit: 4.288051059120335e-06
norm emit: 3.903182459907839e-06
norm emit: 4.01472743760678e-06
norm emit: 4.205948698654538e-06
norm emit: 4.779925802722573e-06
norm emit: 4.654166787076974e-06


In [2]:
# entire stats
# calculate propability of each scale if the rms error is 5%
dist = torch.distributions.Normal(1.0,0.05)
vals = torch.tensor(scales)
probs = dist.log_prob(vals).exp().unsqueeze(0).repeat(5, 1).T

def get_weighted_stats(vals):
    prob_weighted_vals = vals * probs
    weighted_val_mean = prob_weighted_vals.sum() / probs.sum()
    weighted_val_variance = (probs * (emittances - weighted_val_mean)**2).sum() / probs.sum() 
    return weighted_val_mean, weighted_val_variance
    


for name, ele in zip(["emit","beta","alpha"],[emittances*1e6, betas, alphas]):
    mean, variance = get_weighted_stats(ele)
    print(f"mean {name}: {ele.mean()} +/- {ele.std()}")


mean emit: 4.628572463989258 +/- 0.6571033000946045
mean beta: 9.353693962097168 +/- 1.4728230237960815
mean alpha: -0.2349618524312973 +/- 0.23954056203365326


In [3]:
print(emittances.mean(dim=-1)*1e6)
print(emittances.std(dim=-1)*1e6)

tensor([4.8055, 5.1956, 4.1996, 4.6306, 4.3116])
tensor([0.4638, 0.7273, 0.4156, 0.8524, 0.3882])


In [4]:
#for i in range(5):
#    # ensemble rms
#    for name, ele in zip(["emit","beta","alpha"],[emittances[i], betas[i], alphas[i]]):
#        ele = torch.tensor(ele)
#        print(f"{i} {name}: {ele.mean()} +/- {ele.std()}")


In [5]:
import torch
eny = [4.7797, 5.0640, 5.258, 4.7572, 5.0252, 5.2580, 4.9472, 5.1097, 5.4322] # mm mrad 
betay = [8.201, 8.630, 9.083, 8.218, 8.644, 9.083, 7.947, 8.517, 8.775] # m
alphay = [0.097, 0.240, 0.406, 0.101, 0.248, 0.406, 0.085, 0.234, 0.385] # rad
print(f"{torch.tensor(eny).mean()} +/- {torch.tensor(eny).std()}")
print(f"{torch.tensor(betay).mean()} +/- {torch.tensor(betay).std()}")
print(f"{torch.tensor(alphay).mean()} +/- {torch.tensor(alphay).std()}")


5.070133209228516 +/- 0.224358469247818
8.566444396972656 +/- 0.3918430805206299
0.2446666806936264 +/- 0.13221006095409393


## Distribution analysis

In [7]:
for ele in losses:
    plt.semilogy(ele[0].cpu().detach())

AttributeError: 'list' object has no attribute 'cpu'

In [8]:
losses

KeyboardInterrupt: 